## SQL Alchemy run through

In the previous notebook, we made an RDS Instance that contains all of our tables. In this subsection, we will go over retrieving this information using SQL Alchemy

## Making the engine

In [1]:
from sqlalchemy import create_engine, select
from sqlalchemy.dialects.postgresql import psycopg2
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
import psycopg2
import sqlalchemy

In [2]:
import boto3
import pandas as pd

In [3]:
int(sqlalchemy.__version__[0]) >= 1

True

In [4]:
from sqlalchemy.sql import and_, or_

In [5]:
from sqlalchemy import func, text, case

In [6]:
from pprint import pprint

### first get the rds instance we deployed earlier

In [7]:
rds = boto3.client('rds')

In [8]:
def getAddressAndPort(instance_identifier):
    import boto3
    rds_client = boto3.client('rds')

    instance =filter(
        lambda instance: instance['DBInstanceIdentifier'] == instance_identifier,
                           rds.describe_db_instances()['DBInstances'])
    endpoint= list(instance)[0]['Endpoint']
    address, port = endpoint['Address'], endpoint['Port']
    return address, port



In [9]:
getAddressAndPort('cfsdb')

('cfsdb.cs4yyoqmq4gl.us-west-1.rds.amazonaws.com', 5432)

In [10]:
instances = list(filter(lambda instance: instance['DBInstanceStatus']=='available', rds.describe_db_instances()['DBInstances']))

In [11]:
instance = instances[0]

In [12]:
instances

[{'AllocatedStorage': 5,
  'AutoMinorVersionUpgrade': True,
  'AvailabilityZone': 'us-west-1c',
  'BackupRetentionPeriod': 5,
  'CACertificateIdentifier': 'rds-ca-2015',
  'CopyTagsToSnapshot': False,
  'DBInstanceClass': 'db.m4.large',
  'DBInstanceIdentifier': 'cfsdb',
  'DBInstanceStatus': 'available',
  'DBName': 'cfsdb',
  'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres9.4',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSecurityGroups': [],
  'DBSubnetGroup': {'DBSubnetGroupDescription': 'default',
   'DBSubnetGroupName': 'default',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetAvailabilityZone': {'Name': 'us-west-1c'},
     'SubnetIdentifier': 'subnet-1d5ab444',
     'SubnetStatus': 'Active'},
    {'SubnetAvailabilityZone': {'Name': 'us-west-1a'},
     'SubnetIdentifier': 'subnet-e9b1188c',
     'SubnetStatus': 'Active'}],
   'VpcId': 'vpc-bba470de'},
  'DbInstancePort': 0,
  'DbiResourceId': 'db-MRSQSSQK446ID2RK4BSFQ52DWE',
  'Endpoint': {'Address':

In [13]:
address, port = instance['Endpoint']['Address'], instance['Endpoint']['Port']
db_name = instance['DBName']
username = instance['MasterUsername']
password = 'TestPassword'

In [14]:
address, db_name

('cfsdb.cs4yyoqmq4gl.us-west-1.rds.amazonaws.com', 'cfsdb')

In [15]:
engine = create_engine('postgresql+psycopg2://{username}:{password}@{add}:{port}/{db_name}'.format(
    username=username,
    password=password,
    add=address,
    port=port,
    db_name=db_name), echo=True)

In [16]:
conn = engine.connect()

2016-01-15 19:16:27,197 INFO sqlalchemy.engine.base.Engine select version()


INFO:sqlalchemy.engine.base.Engine:select version()


2016-01-15 19:16:27,198 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-01-15 19:16:27,242 INFO sqlalchemy.engine.base.Engine select current_schema()


...

INFO:sqlalchemy.engine.base.Engine:{}


2016-01-15 19:16:27,365 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings


INFO:sqlalchemy.engine.base.Engine:show standard_conforming_strings


2016-01-15 19:16:27,366 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


In [ ]:
# at this point we can remove entry 0 that we used in the earlier notebook
conn.execute("""
             DELETE FROM ONLY transactions
             WHERE "SHIPMT_ID" = 0 """)

In [ ]:
assert conn.execute("""
    SELECT  "SHIPMT_ID" 
    FROM transactions
    LIMIT 1
    """).fetchall()[0][0] == 1, "Failed to delete row"

In [17]:
# first try some basic sql queries
# get hazardous or not hazardous
q = conn.execute("""
             SELECT 
             CASE WHEN "HAZMAT" = 'P' OR "HAZMAT" = 'H' THEN 'Hazardous' 
                  WHEN "HAZMAT" = 'N' THEN 'Not Hazardous' 
                  ELSE  'n/a' END AS "hazmat", 
             COUNT(1) 
             FROM transactions 
             GROUP BY 1
             """
             )

2016-01-15 19:16:32,974 INFO sqlalchemy.engine.base.Engine 
             SELECT 
             CASE WHEN "HAZMAT" = 'P' OR "HAZMAT" = 'H' THEN 'Hazardous' 
                  WHEN "HAZMAT" = 'N' THEN 'Not Hazardous' 
                  ELSE  'n/a' END AS "hazmat", 
             COUNT(1) 
             FROM transactions 
             GROUP BY 1
             


INFO:sqlalchemy.engine.base.Engine:
             SELECT 
             CASE WHEN "HAZMAT" = 'P' OR "HAZMAT" = 'H' THEN 'Hazardous' 
                  WHEN "HAZMAT" = 'N' THEN 'Not Hazardous' 
                  ELSE  'n/a' END AS "hazmat", 
             COUNT(1) 
             FROM transactions 
             GROUP BY 1
             


2016-01-15 19:16:32,976 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


In [18]:
q.fetchall()

[('Hazardous', 302979), ('Not Hazardous', 4244682)]

### Now that you have it working, use SqlAlchemy to show all the tables you currently have

MetaData allows you to store all the information about all the tables in your database in a convenient location you have to use the reflect method of MetaData to accomplish this.

In [19]:
meta = MetaData()

In [20]:
meta.reflect(engine)

2016-01-15 19:16:39,070 INFO sqlalchemy.engine.base.Engine SELECT relname FROM pg_class c WHERE relkind = 'r' AND 'public' = (select nspname from pg_namespace n where n.oid = c.relnamespace) 


INFO:sqlalchemy.engine.base.Engine:SELECT relname FROM pg_class c WHERE relkind = 'r' AND 'public' = (select nspname from pg_namespace n where n.oid = c.relnamespace) 


2016-01-15 19:16:39,071 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-01-15 19:16:39,122 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in ('r', 'v', 'm', 'f')
        


...

INFO:sqlalchemy.engine.base.Engine:{'table_oid': 16442}


2016-01-15 19:16:40,951 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        


INFO:sqlalchemy.engine.base.Engine:
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        


2016-01-15 19:16:40,952 INFO sqlalchemy.engine.base.Engine {'table_oid': 16442}


INFO:sqlalchemy.engine.base.Engine:{'table_oid': 16442}


In [21]:
for table in meta.tables:
    print(table)

transactions_joined
sctg
state_latlon
FIPS_States
naics
cfs_areas
transport_mode
transactions


In [22]:
transactions = meta.tables['transactions']

state_latlon = meta.tables['state_latlon']

cfs_areas    = meta.tables['cfs_areas']
FIPS_States  = meta.tables['FIPS_States']
transport_mode = meta.tables['transport_mode']
sctg         = meta.tables['sctg']
transactions_joined = meta.tables['transactions_joined']

In [23]:
s = select([sctg])
r = conn.execute(s)

2016-01-15 19:16:41,020 INFO sqlalchemy.engine.base.Engine SELECT sctg."SCTG", sctg."Description" 
FROM sctg


INFO:sqlalchemy.engine.base.Engine:SELECT sctg."SCTG", sctg."Description" 
FROM sctg


2016-01-15 19:16:41,021 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


In [24]:
for row in r:
    print(row[sctg.c.Description], row[sctg.c.SCTG])
r.close()

Animals and Fish (live) 1
Cereal Grains (includes seed) 2
Agricultural Products (excludes Animal Feed, Cereal Grains, and Forage Products) 3
Animal Feed, Eggs, Honey, and Other Products of Animal Origin 4
Meat, Poultry, Fish, Seafood, and Their Preparations 5
Milled Grain Products and Preparations, and Bakery Products 6
Other Prepared Foodstuffs, and Fats and Oils 7
Alcoholic Beverages and Denatured Alcohol 8
Tobacco Products 9
Monumental or Building Stone 10
Natural Sands 11
Gravel and Crushed Stone (excludes Dolomite and Slate) 12
Other Non-Metallic Minerals not elsewhere classified 13
Metallic Ores and Concentrates 14
Coal 15
Crude Petroleum 16
Gasoline, Aviation Turbine Fuel, and Ethanol (includes Kerosene, and Fuel Alcohols) 17
Fuel Oils (includes Diesel, Bunker C, and Biodiesel) 18
Other Coal and Petroleum Products, not elsewhere classified 19
Basic Chemicals 20
Pharmaceutical Products 21
Fertilizers 22
Other Chemical Products and Preparations 23
Plastics and Rubber 24
Logs and O

### Joins

In [ ]:
# join fips states with avg lat lon
for row in conn.execute(select([state_latlon, FIPS_States]).where(state_latlon.c.state == FIPS_States.c.get('Alpha code'))):
    print(row)

In [ ]:
FIPS_States.columns.get('Alpha code')

### operators
SQL Alchemy converts python predicates to sql predicates. So you can describe everything in a pythonic sense.

In [ ]:
print((FIPS_States.c.get('Alpha code') == 'KK').compile())
print(((FIPS_States.c.get('Alpha code') < 'CA') & ('NY' < FIPS_States.c.get('Alpha code'))).compile())

In [ ]:
transactions.columns.keys()

In [ ]:
for row in conn.execute(select([transactions]).limit(10)).fetchall():
    print(row.items())

In [ ]:
# to get a list of dicts per row
get_row = lambda row: (dict(row.items()))
get_rows = lambda row_list: map(get_row, row_list)

list_of_dicts = [dict(row.items()) for row in conn.execute(select([transactions]).limit(10)).fetchall()]

### Conjunctions
Conjunctions can be used to perform and, or, not operations in sql.

In [ ]:
# get all states that lie above the 35N, and less than -104 longitude

# first join the fips and state_latlons
j = FIPS_States.join(
    state_latlon, 
    onclause= FIPS_States.c.get("Alpha code") == state_latlon.c.state)

# now select from this join
s = (select([FIPS_States, state_latlon])
     .where(
        and_(
            state_latlon.c.latitude > 36.2,
            state_latlon.c.longitude < -104
        )
    ).select_from(j)
)

In [ ]:
conn.execute(s).fetchall()

In [ ]:
# now get those along with all the states in the bottom right corner of the US
# delinieated by below 35N and right of 88W
s = (select([FIPS_States, state_latlon])
     .where(or_(
        and_(
            state_latlon.c.latitude > 36.2,
            state_latlon.c.longitude < -104
        ),
        and_(
            state_latlon.c.latitude < 35,
            state_latlon.c.longitude > -88)
        )
    ).select_from(j)
)

In [ ]:
pprint(str(s))

In [ ]:
pprint(str(select([text('sub."Alpha code"')]).select_from(s.alias('sub'))))

In [22]:
from sqlalchemy import desc
from sqlalchemy.sql import literal_column

In [ ]:
# using text to select subqueries
# instead of text, we use literal_column so that we can use the label method
conn.execute(select([literal_column('sub."Alpha code"').label("alpha code")]).select_from(s.alias('sub'))
            .order_by(desc('alpha code'))).fetchall()

#### Binding parameters

In [26]:
from sqlalchemy.sql import bindparam, func

In [29]:
get_results = lambda s, **kwargs: conn.execute(s, kwargs).fetchall()

In [25]:
get_results(
    select([FIPS_States])
        .where(or_(
              FIPS_States.c["Alpha code"] == bindparam('from_'),
              FIPS_States.c["Alpha code"] == bindparam('to_')
            )
        ),
    from_ = "CA",
    to_   = "TX"
)

2016-01-09 00:03:20,010 INFO sqlalchemy.engine.base.Engine SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM "FIPS_States" 
WHERE "FIPS_States"."Alpha code" = %(from_)s OR "FIPS_States"."Alpha code" = %(to_)s


INFO:sqlalchemy.engine.base.Engine:SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM "FIPS_States" 
WHERE "FIPS_States"."Alpha code" = %(from_)s OR "FIPS_States"."Alpha code" = %(to_)s


2016-01-09 00:03:20,012 INFO sqlalchemy.engine.base.Engine {'from_': 'CA', 'to_': 'TX'}


INFO:sqlalchemy.engine.base.Engine:{'from_': 'CA', 'to_': 'TX'}


[('California', 'CA', 6, 'State;\xa0counties'),
 ('Texas', 'TX', 48, 'State;\xa0counties')]

#### Functions

Functions are created using the func keyword.  These are translated to sql function queries.

func.myFunc(transactions.c.SHIPMT_WEIGHT) translates to 

    myFunc(transactions."SHIPMT_WEIGHT")

In [ ]:
print(func.myFunc([transactions.c.SHIPMT_VALUE]).compile().params)

In [ ]:
conn.execute(select([func.max(transactions.c.SHIPMT_VALUE).label("Max Val")])).scalar()

#### Window functions
Chain over with functions

In [ ]:
list(get_rows(get_results(select([transactions,
                                 func.sum(transactions.c.SHIPMT_VALUE).over(
                                    order_by=transactions.c.SHIPMT_ID)
                                  .label("SUM")
                                 ]).limit(10))))

In [26]:
from sqlalchemy.sql import union

In [ ]:
u = union(
    sctg.select().where(sctg.c.SCTG < 3),
    sctg.select().where(sctg.c.SCTG > 10)
    )

conn.execute(u).fetchall()

In [ ]:
query_s = (select([func.count(transactions.c.SHIPMT_ID)])
           .where(transactions.c.SCTG).as_scalar())

In [ ]:
conn.execute(select([query_s]))

In [ ]:
conn.execute(u).fetchone()

##### Correlated Subqueries

When an enclosed query depends on a table from the enclosing context, it does not need to explicitly provide a FROM clause

In [ ]:
stmt = (select([transactions, FIPS_States.c['Alpha code']])
        .select_from(
        transactions.join(FIPS_States, onclause=transactions.c.ORIG_STATE == FIPS_States.c["Numeric code"]))
        .where(FIPS_States.c['Alpha code'].like('C%'))
        )

In [ ]:
enclosing_stmt = select([transactions.c.SCTG]).select_from(stmt).limit(10)

In [ ]:
conn.execute(enclosing_stmt)

In [ ]:
transactionsBetweenStates = (select([transactions_joined])
                                 .where(
                transactions_joined.c.orig_alpha_code==bindparam("orig_code"))
                                  .where(
                transactions_joined.c.dest_alpha_code==bindparam("dest_code"))
)


In [ ]:
stmt = (select([transactionsBetweenStates.c['SHIPMT_VALUE']])
.select_from(transactionsBetweenStates.alias())
)



In [ ]:
get_rows(conn.execute(stmt, orig_code="TX", dest_code="WA").fetchall())

##### Controlling correlations

You can use the correlate method to control the behavior of correlations

In [ ]:
import json

Now you can export to a json file for a temporary data file

In [ ]:
#with open('app/testdata.json', 'w') as f:
#    f.write((json.dumps(list_of_dicts, indent=2, separators=(',', ':'))))

In [ ]:
c = case([(transactions.c['ORIG_STATE']==transactions.c['DEST_STATE'], True)], else_=False)

In [ ]:
s = select([transactions]).where(c).limit(10);

In [ ]:
for i in get_rows(conn.execute(s).fetchall()):
    print(i)

Data structure

{orig_state_code:
 dest_state_code:
 orig_coords:
 dest_coords:
}

In [ ]:
joined_states = text(
    """
    SELECT 
        fips."Name" as state_name,
        fips."Numeric code" as numeric_code,
        state_coords.latitude as lat,
        state_coords.longitude as long
    FROM "FIPS_States" fips
    JOIN "state_latlon" state_coords
    ON state_coords.state = fips."Alpha code"
    """)

list(get_rows(conn.execute(joined_states).fetchall()))

In [ ]:
states_and_coords = select([state_latlon, FIPS_States]).where(state_latlon.c.state == FIPS_States.c.get('Alpha code'))

In [ ]:
conn.execute(states_and_coords)

In [ ]:
orig_state = 'CA'
dest_state = 'TX'

In [ ]:
states = select([FIPS_States]).where(or_(
    FIPS_States.c['Alpha code']==orig_state,
    FIPS_States.c['Alpha code']==dest_state)
)

conn.execute(states).fetchall()

### I need a way to query a database like so, 

    "SELECT * 
        FROM transactions
        WHERE orig_state = 'CA' AND dest_state= 'TX'
    "

In [ ]:
d = pd.read_sql_query(con = engine, sql = select([FIPS_States]))

In [ ]:
d2 = pd.read_sql_query(con = engine, sql = select([transactions]).limit(2000))

In [ ]:
d3 = pd.merge(pd.merge(d, d2, left_on=["Numeric code"], right_on=["ORIG_STATE"]), d, 
         left_on="DEST_STATE", right_on="Numeric code")

In [ ]:
d3 = d3.rename(columns={'Name_x': 'Source_Name', 
                   'Alpha code_x': 'Source_alpha_code',
                    'Name_y': 'Dest_Name',
                    'Alpha code_y': 'Dest_alpha_code'})

Use set operators from python to get the intersection of these predicates

In [ ]:
d3[(d3['Dest_alpha_code'] == 'CA') & (d3['Source_alpha_code'] == 'TX')]

In [ ]:
gb = d3.groupby(['Dest_alpha_code', 'Source_alpha_code'])

In [ ]:
gb.get_group(('CA', 'CA')).SHIPMT_VALUE.describe().reset_index()

In [ ]:
gb.groups.keys()

### This way works, but I'm worried that there will be a lot of overhead joining tables constantly, so I will create a new table that contains the source name, and alpha code along with the destination info into a new table called transactions_joined

### You can create a new table from a previous one using sql alchemy's Insert From Select

In [ ]:
#list(get_rows(conn.execute(select([FIPS_States.c.get("Numeric code")])).fetchall()))

In [ ]:
state_latlon_joined = FIPS_States.join(
    state_latlon, onclause=state_latlon.c.state == FIPS_States.c["Alpha code"])

In [ ]:
orig = transactions.join(
    state_latlon_joined, 
    onclause=transactions.c.ORIG_STATE == FIPS_States.c.get("Numeric code"))


s = select([transactions, 
            FIPS_States.c.get("Alpha code").label("orig_state_alpha_code"),
            FIPS_States.c.Name.label("orig_state_name"),
            state_latlon.c.latitude.label('orig_lat'),
            state_latlon.c.longitude.label('orig_lon')
            ]).select_from(orig).limit(10)

list(get_rows(conn.execute(s).fetchall()))

In [ ]:
dest = transactions.join(
    state_latlon_joined, 
    onclause=transactions.c.DEST_STATE == FIPS_States.c.get("Numeric code"))

new_table =  (select([s.alias(), 
            FIPS_States.c.get("Alpha code").label("dest_state_alpha_code"),
            FIPS_States.c.Name.label("dest_state_name"),
            state_latlon.c.latitude.label('dest_lat'),
            state_latlon.c.longitude.label('dest_lon')])
            .select_from(dest).limit(10))

In [ ]:
list(get_rows(conn.execute(select([new_table.alias()])).fetchall()))

In [ ]:
meta.tables['FIPS_States']

In [ ]:
t = text("""
    SELECT transactions.*,
        orig."Name" as orig_name,
        orig."Alpha code" as orig_alpha_code,
        orig."latitude" as orig_lat,
        orig."longitude" as orig_long,
        dest."Name" as dest_name,
        dest."Alpha code" as dest_alpha_code,
        dest."latitude" as dest_lat,
        dest."longitude" as dest_lon
    FROM transactions
    JOIN (
        SELECT states.*, lonlats.*
        FROM "FIPS_States" states
        JOIN state_latlon lonlats
        on states."Alpha code"=lonlats.state
         ) orig
        ON orig."Numeric code" = transactions."ORIG_STATE"
    JOIN (
        SELECT states.*, lonlats.*
        FROM "FIPS_States" states
        JOIN state_latlon lonlats
        on states."Alpha code"=lonlats.state
         ) dest
        ON dest."Numeric code" = transactions."DEST_STATE"
    WHERE
        dest."Alpha code"=:dest_code AND orig."Alpha code" =:orig_code
    LIMIT 100
        
    """)

In [ ]:
select([transactions,
        ])

In [ ]:
conn.execute(t.params(dest_code="CA", orig_code="TX")).fetchone()

In [ ]:
#with open('test_data.json', 'w') as f:
#    f.write(
#        json.dumps(
#            list(get_rows(conn.execute(t).fetchall())),
#            indent=2,
#            separators=(",", ":")
#             ))

In [27]:
from sqlalchemy.sql import table

In [ ]:
#new_table = new_table.alias("namedStates")

In [ ]:
new_table_t = table(new_table)

In [ ]:
new_tableb

In [ ]:
orig_ = new_table.join(
    state_latlon,
    onclause=state_latlon.c.state == new_table.c.orig_state_alpha_code)

dest_ = new_table.join(
    state_latlon,
    onclause=state_latlon.c.state == new_table.c.dest_state_alpha_code)

In [ ]:
new_table2 = (select([new_table.alias(),
                    state_latlon.c.latitude.label('orig_lat'),
                    state_latlon.c.longitude.label('orig_lon')])
            .select_from(orig_.alias()))

In [ ]:
get_rows(conn.execute(select([new_table2.alias()])))

In [ ]:
list(get_rows(conn.execute(select([state_latlon])).fetchall()))

In [ ]:
list(get_rows(conn.execute(select([FIPS_States])).fetchall()))

In [ ]:
orig = state_latlon_joined.alias("orig")

dest = state_latlon_joined.alias("dest")
   

In [ ]:
orig.c.keys()

In [28]:
from sqlalchemy.sql import bindparam

In [ ]:
state_latlon_joined = FIPS_States.join(
    state_latlon, onclause=state_latlon.c.state == FIPS_States.c["Alpha code"])

stmt = (select([transactions,
              orig.c.FIPS_States_Name.label("orig_name"),
              orig.c["FIPS_States_Alpha code"].label("orig_alpha_code"),
              dest.c["FIPS_States_Name"].label("dest_name"),
              dest.c["FIPS_States_Alpha code"].label("dest_alpha_code"),
              orig.c.state_latlon_latitude.label("orig_lat"),
              orig.c.state_latlon_longitude.label("orig_lon"),
              dest.c.state_latlon_latitude.label("dest_lat"),
              dest.c.state_latlon_longitude.label("dest_lon")])
        .select_from(transactions
                    .join(orig, transactions.c.ORIG_STATE == orig.c["FIPS_States_Numeric code"])
                    .join(dest, transactions.c.DEST_STATE == dest.c["FIPS_States_Numeric code"]))
        )
        #.where(orig.c["FIPS_States_Alpha code"] == bindparam("orig_code")))
        #.where(dest.c["FIPS_States_Alpha code"] == bindparam("dest_code")))

In [ ]:
list(get_rows(conn.execute(stmt.order_by(desc(transactions.c.SHIPMT_VALUE)).limit(10), orig_code='CA').fetchall()))

Create table from another table


In [ ]:
stmt.columns.items()

In [ ]:
# a way to copy a columns from one table to another
t = Table("transactions_joined", meta, *(map(lambda c: c.copy(), stmt.columns)))


In [ ]:
t.create(bind=engine)

In [ ]:
# pass in the column keys and the statement to select from
conn.execute(t.insert().from_select(stmt.columns.keys(), stmt))

In [ ]:
meta.reflect(engine)

In [ ]:
meta.tables.keys()

In [ ]:
tj = meta.tables['transactions_joined']

In [ ]:
q = (select([tj])
     .where(tj.c.orig_alpha_code==bindparam("orig_code"))
     .where(tj.c.dest_alpha_code==bindparam("dest_code"))
     .order_by(tj.c.SHIPMT_VALUE)
     .limit(10))

q2 = (select([tj])
     .where(tj.c.orig_alpha_code==bindparam("orig_code"))
      .order_by(desc(tj.c.SHIPMT_VALUE))
      .limit(10))
list(get_rows(conn.execute(q, orig_code="CA", dest_code="TX")))

In [ ]:
tj.columns.keys()

In [ ]:
from sqlalchemy import Index

Index('orig_fip', transactions.c.ORIG_STATE)

In [ ]:
import timeit

I want to experiment with adding an index and testing the response of the query

In [ ]:
import time

In [ ]:
print("We will be running: {}".format(str(q)))

We need to make an index on the orig_alpha_code and dest_alpha_code

In [ ]:
index = text("CREATE INDEX ALPHA_INDEX ON transactions_joined "\
             "(orig_alpha_code, dest_alpha_code)")

In [ ]:
conn.execute(index)

In [ ]:
meta.tables['transactions_joined']

In [29]:
#http://stackoverflow.com/questions/5605019/listing-indices-using-sqlalchemy
from sqlalchemy.engine import reflection

In [30]:
insp = reflection.Inspector.from_engine(engine)
indices = []
for name in insp.get_table_names():
    for index in insp.get_indexes(name):
        indices.append((name,index))

2016-01-09 00:05:17,651 INFO sqlalchemy.engine.base.Engine SELECT relname FROM pg_class c WHERE relkind = 'r' AND 'public' = (select nspname from pg_namespace n where n.oid = c.relnamespace) 


INFO:sqlalchemy.engine.base.Engine:SELECT relname FROM pg_class c WHERE relkind = 'r' AND 'public' = (select nspname from pg_namespace n where n.oid = c.relnamespace) 


2016-01-09 00:05:17,653 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-01-09 00:05:17,789 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in ('r', 'v', 'm', 'f')
        


...

INFO:sqlalchemy.engine.base.Engine:{'table_name': 'naics'}


2016-01-09 00:05:19,766 INFO sqlalchemy.engine.base.Engine 
              SELECT
                  i.relname as relname,
                  ix.indisunique, ix.indexprs, ix.indpred,
                  a.attname, a.attnum, c.conrelid, ix.indkey::varchar,
                  i.reloptions, am.amname
              FROM
                  pg_class t
                        join pg_index ix on t.oid = ix.indrelid
                        join pg_class i on i.oid = ix.indexrelid
                        left outer join
                            pg_attribute a
                            on t.oid = a.attrelid and a.attnum = ANY(ix.indkey)
                        left outer join
                            pg_constraint c
                            on (ix.indrelid = c.conrelid and
                                ix.indexrelid = c.conindid and
                                c.contype in ('p', 'u', 'x'))
                        left outer join
                            pg_am am
                    

INFO:sqlalchemy.engine.base.Engine:
              SELECT
                  i.relname as relname,
                  ix.indisunique, ix.indexprs, ix.indpred,
                  a.attname, a.attnum, c.conrelid, ix.indkey::varchar,
                  i.reloptions, am.amname
              FROM
                  pg_class t
                        join pg_index ix on t.oid = ix.indrelid
                        join pg_class i on i.oid = ix.indexrelid
                        left outer join
                            pg_attribute a
                            on t.oid = a.attrelid and a.attnum = ANY(ix.indkey)
                        left outer join
                            pg_constraint c
                            on (ix.indrelid = c.conrelid and
                                ix.indexrelid = c.conindid and
                                c.contype in ('p', 'u', 'x'))
                        left outer join
                            pg_am am
                            on i.relam = am.

2016-01-09 00:05:19,767 INFO sqlalchemy.engine.base.Engine {'table_oid': 16442}


INFO:sqlalchemy.engine.base.Engine:{'table_oid': 16442}


In [31]:
indices

[('transactions_joined',
  {'column_names': ['SHIPMT_VALUE'],
   'name': 'shipmt_value_desc',
   'unique': False}),
 ('transactions_joined',
  {'column_names': ['SHIPMT_VALUE'], 'name': 'shipmt_value', 'unique': False}),
 ('transactions_joined',
  {'column_names': ['orig_alpha_code', 'dest_alpha_code'],
   'name': 'alpha_index',
   'unique': False})]

In [ ]:
import datetime

In [ ]:
now = datetime.datetime.now()
l = list(get_rows(conn.execute(q, orig_code="NY", dest_code="TX").fetchall()))
diff = datetime.datetime.now() - now



In [ ]:
print("It took {} seconds to execute the query".format(diff))
pprint(l)

In [32]:
#http://docs.sqlalchemy.org/en/latest/faq/performance.html#query-profiling
from sqlalchemy import event
from sqlalchemy.engine import Engine

In [33]:
@event.listens_for(Engine, "before_cursor_execute")
def before_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    conn.info.setdefault('query_start_time', []).append(time.time())
    print("Start Query: {}".format(statement))

In [34]:
@event.listens_for(Engine, "after_cursor_execute")
def after_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    total = time.time() - conn.info['query_start_time'].pop(-1)
    print("Query Done\nTotal-Time: {}".format(total))

In [ ]:
l = list(get_rows(conn.execute(q2, orig_code="CA").fetchall()))

In [ ]:
cmp = q.params(orig_code="CA", dest_code="TX").compile()

In [ ]:
print(cmp)

In [ ]:
# try creating a new index on shipmt_value
i = Index('shipmt_value', tj.c.SHIPMT_VALUE)
i.create(engine)

In [ ]:
l = list(get_rows(conn.execute(q, orig_code="WY", dest_code="WY").fetchall()))

In [ ]:
# that index made it much faster
# let's try adding a functional index as well, a desc version of shipmt_value
i = Index('shipmt_value_desc', tj.c.SHIPMT_VALUE.desc())
i.create(engine)

### Adding indexes greatly reduces time!

## Now I will be figuring out the queries needed for the other data views

In [ ]:
# we want to get the sctg names joined
q = (select([tj.c.SCTG, func.count(tj.c.SHIPMT_ID).label("counts")])
     .where(and_(
        tj.c.dest_alpha_code == 'TX',
        tj.c.orig_alpha_code == 'WY'))
     )
        

In [ ]:
g1 = q.group_by(tj.c.SCTG)

In [ ]:
for i in get_rows(conn.execute(g1.limit(10)).fetchall()):
    print(i)

In [ ]:
sctg.columns.keys()

In [ ]:
for i in get_rows(conn.execute(select([g1.alias(), sctg]).where(tj.c.SCTG == cast(sctg.c.SCTG, TEXT()))).fetchall()):
    print(i)

In [ ]:
from sqlalchemy import cast, Numeric, String, TEXT

In [ ]:
tj.c.SCTG, print(cast(sctg.c.SCTG, TEXT()))

In [27]:
transactionsBetweenStates = (select([transactions_joined])
                                 .where(
                transactions_joined.c.orig_alpha_code==bindparam("orig_code"))
                                  .where(
                transactions_joined.c.dest_alpha_code==bindparam("dest_code"))
)




In [28]:
q = transactionsBetweenStates.params(orig_code='CA', dest_code='CA')

In [34]:
_data = pd.read_sql_query(q, engine)

2016-01-15 19:22:55,809 INFO sqlalchemy.engine.base.Engine SELECT transactions_joined."SHIPMT_ID", transactions_joined."ORIG_STATE", transactions_joined."ORIG_MA", transactions_joined."ORIG_CFS_AREA", transactions_joined."DEST_STATE", transactions_joined."DEST_MA", transactions_joined."DEST_CFS_AREA", transactions_joined."NAICS", transactions_joined."QUARTER", transactions_joined."SCTG", transactions_joined."MODE", transactions_joined."SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED", transactions_joined."TEMP_CNTL_YN", transactions_joined."EXPORT_YN", transactions_joined."EXPORT_CNTRY", transactions_joined."HAZMAT", transactions_joined."WGT_FACTOR", transactions_joined.orig_name, transactions_joined.orig_alpha_code, transactions_joined.dest_name, transactions_joined.dest_alpha_code, transactions_joined.orig_lat, transactions_joined.orig_lon, transactions_joined.dest_lat, transactions_joined.dest_lon 
FROM 

INFO:sqlalchemy.engine.base.Engine:SELECT transactions_joined."SHIPMT_ID", transactions_joined."ORIG_STATE", transactions_joined."ORIG_MA", transactions_joined."ORIG_CFS_AREA", transactions_joined."DEST_STATE", transactions_joined."DEST_MA", transactions_joined."DEST_CFS_AREA", transactions_joined."NAICS", transactions_joined."QUARTER", transactions_joined."SCTG", transactions_joined."MODE", transactions_joined."SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED", transactions_joined."TEMP_CNTL_YN", transactions_joined."EXPORT_YN", transactions_joined."EXPORT_CNTRY", transactions_joined."HAZMAT", transactions_joined."WGT_FACTOR", transactions_joined.orig_name, transactions_joined.orig_alpha_code, transactions_joined.dest_name, transactions_joined.dest_alpha_code, transactions_joined.orig_lat, transactions_joined.orig_lon, transactions_joined.dest_lat, transactions_joined.dest_lon 
FROM transactions_joined 
WHE

2016-01-15 19:22:55,811 INFO sqlalchemy.engine.base.Engine {'dest_code': 'CA', 'orig_code': 'CA'}


INFO:sqlalchemy.engine.base.Engine:{'dest_code': 'CA', 'orig_code': 'CA'}


In [51]:
_data['QUARTER'].value_counts

Object `value_counts` not found.


In [56]:
sser= _data['QUARTER'].value_counts(

In [61]:
import json

In [68]:
json.dumps([{str(k):str(v)} for k,v in sser.sort_index().to_dict().items()])

'[{"1": "58524"}, {"2": "58017"}, {"3": "55253"}, {"4": "50051"}]'